<a href="https://colab.research.google.com/github/wvls123/colab/blob/main/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_18_%D0%BD%D0%BE%D1%8F%D0%B1%D1%80%D1%8F_Weweb_%2B_GPT_%D0%94%D0%B5%D0%BD%D1%8C_1_%D0%9A%D0%BE%D0%BB%D0%B0%D0%B1_%D0%B0%D1%81%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BD%D1%82%D0%B0_%D0%BF%D0%BE_%D0%B1%D1%80%D0%B5%D0%BD%D0%B4%D1%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Рабочий от VC  gpt (с пингами)

In [ ]:
# ==========================================================
#  NeuroAnalyst Backend FINAL STABLE VERSION
# ==========================================================
#  ✔ Парсинг сайта (устойчивый)
#  ✔ Основной анализ модели (через Google Doc)
#  ✔ Follow-up анализ (через другой Google Doc)
#  ✔ Follow-up НЕ получает site_data
#  ✔ Единый JSON вход для follow-up
#  ✔ История в формате role/content
#  ✔ Cloudflared туннель
#  ✔ CORS для WeWeb
#  ✔ Исправлены ошибки link undefined, mailto, tel, javascript
#  ✔ Ограничение страниц depth=1, max_pages=15
# ==========================================================


!pip install openai flask flask-cors beautifulsoup4 requests tldextract >/dev/null
import os, re, json, time, threading, subprocess, urllib.request, uuid, logging
from datetime import datetime, timedelta
from flask import Flask, request, jsonify
from flask_cors import CORS
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from getpass import getpass
import tldextract
import requests
from openai import OpenAI
# -------------------------------
# ⚙️ Конфигурация
# -------------------------------
PORT = 8000
CLOUDFLARED_PATH = "/usr/local/bin/cloudflared"
MAIN_PROMPT_URL = "https://docs.google.com/document/d/1DtA6CzcNeoZSDwj043YmE84XMnv1LAp_Z3MWxP8n55M/edit"
FOLLOWUP_PROMPT_URL = "https://docs.google.com/document/d/12nwxCLf4Gk4daR7ecRA04rZe-RToNb8-TAtERzY4o0E/edit"
SESSION_TTL_HOURS = 24
MAX_SESSIONS = 100
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger("neuro-analyst")
if not os.path.exists(CLOUDFLARED_PATH):
    print("🔽 Устанавливаю cloudflared...")
    url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
    urllib.request.urlretrieve(url, "cloudflared")
    os.rename("cloudflared", CLOUDFLARED_PATH)
    os.chmod(CLOUDFLARED_PATH, 0o755)
else:
    print("✅ Cloudflared уже установлен.")
# -------------------------------
# 📄 ИСПРАВЛЕНО: Загрузка текста из Google Docs
# -------------------------------
def fetch_gdoc_text(gdoc_url: str) -> str:
    try:
        # Преобразуем URL в формат экспорта текста
        if '/edit' in gdoc_url:
            export_url = gdoc_url.replace('/edit', '/export?format=txt')
        elif '/view' in gdoc_url:
            export_url = gdoc_url.replace('/view', '/export?format=txt')
        else:
            # Если URL уже в другом формате, добавляем экспорт
            export_url = gdoc_url.rstrip('/') + '/export?format=txt'

        logger.info(f"📥 Загружаю документ: {export_url}")

        r = requests.get(export_url)
        if r.status_code != 200:
            raise ValueError(f"Ошибка загрузки Google Doc: {r.status_code}")

        text = r.text.strip()

        # Проверяем, что получили текст, а не HTML
        if text.startswith('<!DOCTYPE') or text.startswith('<html'):
            raise ValueError("Получен HTML вместо текста. Проверьте, что документ публично доступен")

        logger.info(f"📄 Загружен Google Doc")
        logger.info(f"📊 Размер: {len(text):,} символов (~{len(text)//4:,} токенов)")

        return text
    except Exception as e:
        raise ValueError(f"Ошибка загрузки документа: {e}")
# -------------------------------
# 🌐 Фильтрация и нормализация ссылок
# -------------------------------
def normalize_link(base, href: str):
    if not href or not isinstance(href, str):
        return None
    href = href.strip()
    bad_prefixes = (
        "mailto:", "tel:", "javascript:", "whatsapp:", "viber:",
        "tg:", "#", "sms:", "skype:",
    )
    if href.startswith(bad_prefixes):
        return None
    if href.startswith("http://") or href.startswith("https://"):
        return href.split("#")[0]
    if href.startswith("//"):
        return "https:" + href.split("#")[0]
    return urljoin(base, href.split("#")[0])
# -------------------------------
# 🔎 Парсинг сайта
# -------------------------------
def same_domain(a, b):
    try:
        return tldextract.extract(a).registered_domain == tldextract.extract(b).registered_domain
    except:
        return False
def safe_json(obj):
    try:
        json.dumps(obj)
        return obj
    except:
        return str(obj)
def crawl_site(start_url, max_pages=15, depth=1):
    logger.info(f"🔎 Начинаю парсинг: {start_url}")
    logger.info(f"⚙️ Настройки: max_pages={max_pages}, depth={depth}")

    visited, queue = set(), [(start_url, 0)]
    pages = []
    total_text_chars = 0
    while queue and len(pages) < max_pages:
        url, d = queue.pop(0)
        if url in visited or d > depth:
            continue
        visited.add(url)
        logger.info(f"🌐 Парсим [{len(pages)+1}/{max_pages}]: {url}")
        try:
            r = requests.get(url, timeout=8, headers={"User-Agent": "NeuroAnalystBot/1.0"})
            if r.status_code != 200:
                logger.warning(f"⚠️ Статус {r.status_code}, пропускаю")
                continue
            soup = BeautifulSoup(r.text, "html.parser")
            for s in soup(["script", "style", "noscript"]):
                s.extract()
            title = soup.title.string.strip() if soup.title else ""
            text = soup.get_text("\n", strip=True)

            original_len = len(text)
            text = text[:20000]
            total_text_chars += len(text)
            meta = {
                m.get("name", m.get("property", "")): m.get("content", "")
                for m in soup.find_all("meta")
                if m.get("name") or m.get("property")
            }
            links = []
            for a in soup.find_all("a", href=True):
                link = normalize_link(url, a["href"])
                if not link:
                    continue
                if same_domain(start_url, link):
                    links.append(link)
            page_data = {
                "url": url,
                "title": title,
                "meta": safe_json(meta),
                "text": text,
                "links": links
            }

            page_json = json.dumps(page_data, ensure_ascii=False)
            page_json_len = len(page_json)

            logger.info(f" ├─ Заголовок: {title[:50]}...")
            logger.info(f" ├─ Текст: {original_len:,} → {len(text):,} символов")
            logger.info(f" ├─ JSON: {page_json_len:,} символов (~{page_json_len//4:,} токенов)")
            logger.info(f" └─ Ссылок: {len(links)}")

            pages.append(page_data)
            for l in links:
                if l not in visited:
                    queue.append((l, d + 1))
        except Exception as e:
            logger.error(f"⚠️ Ошибка парсинга {url}: {e}")
    result = {"start_url": start_url, "pages": pages, "count": len(pages)}
    result_json = json.dumps(result, ensure_ascii=False)
    result_json_len = len(result_json)

    logger.info("=" * 60)
    logger.info(f"✅ ПАРСИНГ ЗАВЕРШЁН")
    logger.info(f"📊 Страниц: {len(pages)}")
    logger.info(f"📊 Текста: {total_text_chars:,} символов")
    logger.info(f"📊 JSON: {result_json_len:,} символов (~{result_json_len//4:,} токенов)")
    logger.info("=" * 60)

    return result
# -------------------------------
# 🤖 Основная модель с жёсткой JSON схемой
# -------------------------------
def call_main_model(client, prompt_text, site_data):
    logger.info("🤖 Подготовка запроса к GPT...")

    user_content = json.dumps({"site": site_data}, ensure_ascii=False)

    # Добавляем строгие инструкции к промпту
    enhanced_prompt = prompt_text + """

КРИТИЧЕСКИ ВАЖНО - СТРОГИЕ ОГРАНИЧЕНИЯ:

ИСПОЛЬЗУЙ ТОЛЬКО ЭТИ ПОЛЯ:
{
  "reasoning": {"goals": "...", "dialog_stage": "...", "plan": [...]},
  "analysis": {"archetypes": {...}, "summary": "..."},
  "insights": {
    "strengths": [...],
    "weaknesses": [...],
    "recommendations": {"strengthen": [...], "balance": [...], "add": [...]},
    "overall_comment": "..."
  },
  "strategies": [...],
  "chat_summary": "...",
  "next_question": "..."
}

ЗАПРЕЩЕНО добавлять: notes_on_source, recommendations_for_action, data_quality, assumptions.

ВАЖНО: Не упоминай в тексте "по шагу X из плана", "согласно плану", "на шаге N".
Reasoning - это твой внутренний план, пользователь его не видит.
Пиши естественно, без ссылок на техническую структуру.
"""

    messages = [
        {"role": "system", "content": enhanced_prompt},
        {"role": "user", "content": user_content},
    ]

    system_len = len(enhanced_prompt)
    user_len = len(user_content)
    total_len = system_len + user_len

    logger.info(f"📊 System: {system_len:,} симв. (~{system_len//4:,} токенов)")
    logger.info(f"📊 User: {user_len:,} симв. (~{user_len//4:,} токенов)")
    logger.info(f"📊 ИТОГО: {total_len:,} симв. (~{total_len//4:,} токенов)")
    logger.info(f"🚀 Запрос к gpt-5-mini...")

    resp = client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages,
        response_format={"type": "json_object"}
    )

    output_text = resp.choices[0].message.content
    output_len = len(output_text)

    logger.info(f"✅ Ответ получен")
    logger.info(f"📊 Output: {output_len:,} симв. (~{output_len//4:,} токенов)")
    logger.info(f"📊 API токены: input={resp.usage.prompt_tokens:,}, output={resp.usage.completion_tokens:,}, total={resp.usage.total_tokens:,}")

    return resp
# -------------------------------
# 🤖 Follow-up модель
# -------------------------------
def call_followup_model(client, followup_prompt_text, json_payload):
    logger.info("💬 Follow-up запрос...")

    user_content = json.dumps(json_payload, ensure_ascii=False)

    messages = [
        {"role": "system", "content": followup_prompt_text},
        {"role": "user", "content": user_content},
    ]

    system_len = len(followup_prompt_text)
    user_len = len(user_content)
    total_len = system_len + user_len

    logger.info(f"📊 System: {system_len:,} симв. (~{system_len//4:,} токенов)")
    logger.info(f"📊 User: {user_len:,} симв. (~{user_len//4:,} токенов)")
    logger.info(f"📊 ИТОГО: {total_len:,} симв. (~{total_len//4:,} токенов)")
    logger.info(f"🚀 Запрос к gpt...")

    resp = client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
        response_format={"type": "json_object"}
    )

    output_text = resp.choices[0].message.content
    output_len = len(output_text)

    logger.info(f"✅ Ответ получен")
    logger.info(f"📊 Output: {output_len:,} симв. (~{output_len//4:,} токенов)")
    logger.info(f"📊 API токены: input={resp.usage.prompt_tokens:,}, output={resp.usage.completion_tokens:,}, total={resp.usage.total_tokens:,}")

    return resp
# -------------------------------
# 🗄️ Управление сессиями
# -------------------------------
STORE = {}
def cleanup_old_sessions():
    now = datetime.now()
    to_delete = []

    for sid, sess in STORE.items():
        created_at = sess.get("created_at")
        if created_at and (now - created_at) > timedelta(hours=SESSION_TTL_HOURS):
            to_delete.append(sid)

    for sid in to_delete:
        del STORE[sid]
        logger.info(f"🗑️ Удалена старая сессия: {sid}")

    if to_delete:
        logger.info(f"🧹 Очищено {len(to_delete)} старых сессий")
def limit_sessions():
    if len(STORE) > MAX_SESSIONS:
        sorted_sessions = sorted(
            STORE.items(),
            key=lambda x: x[1].get("created_at", datetime.min)
        )

        to_delete = len(STORE) - MAX_SESSIONS
        for sid, _ in sorted_sessions[:to_delete]:
            del STORE[sid]
            logger.info(f"🗑️ Удалена сессия (лимит): {sid}")

        logger.info(f"🧹 Очищено {to_delete} сессий (лимит достигнут)")
# -------------------------------
# 🌍 Flask API
# -------------------------------
app = Flask(__name__)
CORS(app, resources={
    r"/*": {
        "origins": "*",
        "methods": ["GET", "POST", "OPTIONS"],
        "allow_headers": ["Content-Type"]
    }
})
@app.route("/ping", methods=["GET"])
def ping():
    logger.info("🟢 Ping")
    return jsonify({
        "status": "alive",
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "sessions": len(STORE)
    }), 200
@app.route("/analyze", methods=["POST"])
def analyze():
    logger.info("=" * 60)
    logger.info("🆕 Запрос /analyze")

    cleanup_old_sessions()
    limit_sessions()

    data = request.json or {}
    site_url = data.get("site_url")
    existing_sid = data.get("session_id")
    if not site_url:
        return jsonify({"error": "Нужно указать site_url"}), 400
    logger.info(f"🌐 URL: {site_url}")

    if existing_sid and existing_sid in STORE:
        sid = existing_sid
        logger.info(f"♻️ Переиспользую session_id: {sid}")
        logger.info(f"🗑️ Очищаю старые данные сессии...")
    else:
        sid = str(uuid.uuid4())
        logger.info(f"🆕 Создаю новый session_id: {sid}")
    try:
        main_prompt = fetch_gdoc_text(MAIN_PROMPT_URL)
        site_data = crawl_site(site_url)
        resp = call_main_model(OPENAI_CLIENT, main_prompt, site_data)
        model_output = resp.choices[0].message.content
    except Exception as e:
        logger.error(f"❌ ОШИБКА: {e}")
        return jsonify({"error": str(e)}), 500
    STORE[sid] = {
        "site": site_data,
        "first_output": model_output,
        "last_followup": None,
        "history": [],
        "created_at": datetime.now()
    }
    logger.info(f"✅ Анализ завершён")
    logger.info(f"📊 Активных сессий: {len(STORE)}")
    logger.info("=" * 60)
    return jsonify({
        "session_id": sid,
        "result": model_output,
        "pages": site_data["count"]
    })
@app.route("/followup", methods=["POST"])
def followup():
    logger.info("=" * 60)
    logger.info("💬 Запрос /followup")

    data = request.json or {}
    sid = data.get("session_id")
    user_instruction = data.get("followup_prompt")
    if not sid or sid not in STORE:
        return jsonify({"error": "session_id не найден"}), 404
    logger.info(f"📝 Session: {sid}")
    logger.info(f"💬 Вопрос: {user_instruction[:100]}...")
    sess = STORE[sid]
    try:
        followup_prompt_text = fetch_gdoc_text(FOLLOWUP_PROMPT_URL)
    except Exception as e:
        logger.error(f"❌ ОШИБКА загрузки промпта: {e}")
        return jsonify({"error": f"Ошибка загрузки follow-up промпта: {e}"}), 500
    payload = {
        "first_output": sess.get("first_output"),
        "last_followup": sess.get("last_followup"),
        "conversation_history": sess.get("history", []),
        "user_instruction": user_instruction
    }
    try:
        resp = call_followup_model(OPENAI_CLIENT, followup_prompt_text, payload)
        model_text = resp.choices[0].message.content
        sess["last_followup"] = model_text
        sess["history"].append({"role": "user", "content": user_instruction})
        sess["history"].append({"role": "assistant", "content": model_text})
    except Exception as e:
        logger.error(f"❌ ОШИБКА: {e}")
        return jsonify({"error": str(e)}), 500
    logger.info(f"✅ Follow-up завершён")
    logger.info(f"📊 Сообщений в истории: {len(sess['history'])}")
    logger.info("=" * 60)
    return jsonify({
        "result": model_text
    })
@app.route("/clear-chat", methods=["POST"])
def clear_chat():
    logger.info("🧹 Запрос /clear-chat")

    data = request.json or {}
    sid = data.get("session_id")
    if not sid or sid not in STORE:
        return jsonify({"error": "session_id не найден"}), 404
    sess = STORE[sid]
    messages_count = len(sess.get("history", []))

    sess["history"] = []
    sess["last_followup"] = None

    logger.info(f"✅ Очищена история чата для {sid}")
    logger.info(f"📊 Удалено сообщений: {messages_count}")
    logger.info(f"📊 Результат анализа сохранён")
    return jsonify({
        "status": "success",
        "message": f"История чата очищена ({messages_count} сообщений удалено)",
        "session_id": sid
    }), 200
def run_flask():
    app.run(host="0.0.0.0", port=PORT, use_reloader=False)
# -------------------------------
# 🚀 Запуск
# -------------------------------
print("🔐 Введите OpenAI API ключ:")
OPENAI_API_KEY = getpass("API Key: ")
OPENAI_CLIENT = OpenAI(api_key=OPENAI_API_KEY)
threading.Thread(target=run_flask, daemon=True).start()
print(f"⚙️ Flask на порту {PORT}")
print("🌐 Запуск cloudflared...")
proc = subprocess.Popen(
    [CLOUDFLARED_PATH, "tunnel", "--url", f"http://localhost:{PORT}", "--no-autoupdate"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)
public_url = None
for _ in range(60):
    line = proc.stdout.readline()
    if "trycloudflare.com" in line:
        m = re.search(r"(https://[^\s]+trycloudflare\.com)", line)
        if m:
            public_url = m.group(1)
            break
if public_url:
    print("=" * 60)
    print("✅ СЕРВЕР ЗАПУЩЕН!")
    print("=" * 60)
    print(f"🌐 URL: {public_url}")
    print(f"\n📌 POST {public_url}/analyze")
    print(f"📌 POST {public_url}/followup")
    print(f"📌 POST {public_url}/clear-chat")
    print(f"🟢 GET {public_url}/ping")
    print("=" * 60)
else:
    print("⚠️ Не удалось получить URL.")
print("\n🟢 Colab активен")
print("💡 Настрой пинги из WeWeb на /ping каждые 5 минут")
while True:
    time.sleep(60)

🔽 Устанавливаю cloudflared...
🔐 Введите OpenAI API ключ:
API Key: ··········
⚙️ Flask на порту 8000
🌐 Запуск cloudflared...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit


✅ СЕРВЕР ЗАПУЩЕН!
🌐 URL: https://summit-configuration-tournament-avatar.trycloudflare.com

📌 POST https://summit-configuration-tournament-avatar.trycloudflare.com/analyze
📌 POST https://summit-configuration-tournament-avatar.trycloudflare.com/followup
📌 POST https://summit-configuration-tournament-avatar.trycloudflare.com/clear-chat
🟢 GET https://summit-configuration-tournament-avatar.trycloudflare.com/ping

🟢 Colab активен
💡 Настрой пинги из WeWeb на /ping каждые 5 минут


INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 17:01:16] "OPTIONS /analyze HTTP/1.1" 200 -
/tmp/ipython-input-1757423248.py:106: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  return tldextract.extract(a).registered_domain == tldextract.extract(b).registered_domain
INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 17:02:10] "OPTIONS /ping HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 17:02:10] "GET /ping HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 17:02:27] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 17:06:11] "OPTIONS /ping HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 17:06:11] "GET /ping HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 17:10:12] "OPTIONS /ping HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2025 17:10:12] "GET /ping HTTP/1.1" 200 -
INFO:werk

KeyboardInterrupt: 